# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities=pd.read_csv("../WeatherPy/cities.csv")
cities

,City,Max Temp,Cloudiness,Latitude,Longitude,Humidity,Wind Speed,Country,Date
0,punta arenas,69.80,0,-53.1500,-70.9167,52,5.75,CL,1612904192
1,vila velha,78.01,90,-20.3297,-40.2925,94,5.75,BR,1612904363
2,bredasdorp,64.40,11,-34.5322,20.0403,82,2.30,ZA,1612904363
3,meulaboh,77.59,15,4.1363,96.1285,85,4.59,ID,1612904363
4,vaini,80.60,90,-21.2000,-175.2000,89,21.85,TO,1612904364
...,...,...,...,...,...,...,...,...,...
545,puerto madryn,75.31,100,-42.7692,-65.0385,34,23.76,AR,1612904516
546,mednogorsk,-19.28,62,51.4128,57.5950,87,3.53,RU,1612904517
547,bouloupari,77.00,75,-21.8667,166.0500,88,2.30,NC,1612904517
548,san policarpo,78.73,100,12.1791,125.5072,86,12.97,PH,1612904517


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=cities[['Latitude','Longitude']]
hum=cities['Humidity'].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=hum)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_df=cities.loc[(cities['Max Temp'] <80) & (cities['Max Temp']>70)]
new_df=new_df.loc[new_df['Wind Speed']<10]
ideal_weather_df=new_df.loc[new_df['Cloudiness']==0]
ideal_weather_df=ideal_weather_df.loc[new_df['Humidity']>20]
ideal_weather_df

,City,Max Temp,Cloudiness,Latitude,Longitude,Humidity,Wind Speed,Country,Date
38,east london,72.03,0,-33.0153,27.9116,77,3.96,ZA,1612904371
72,salalah,75.20,0,17.0151,54.0924,60,1.14,OM,1612904380
75,olavarria,75.00,0,-36.8927,-60.3225,45,5.99,AR,1612904380
154,chuy,72.46,0,-33.6971,-53.4616,75,9.10,UY,1612904394
415,broken hill,71.60,0,-31.9500,141.4333,28,6.91,AU,1612904491


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=ideal_weather_df

In [9]:
hotel_df['Hotel Name']=""
params={"radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key}
for index, row in hotel_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url, params=params)
    response=response.json()
    print(json.dumps(response, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, "Hotel Name"]=response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Error.")
hotel_df.info()

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKFcQ4wyUM2rlONMIYvW39R4zVFEDhHeIg0SUK5tzLQBRuWXrobnBZ9aKVMLwK21hJluVq3XVz0i58-sDPCmpWFfd9G80AAUMesJCGRAdm3aYjyYw65rhpeSoYmJWMN0zgLRibxETcDCmSdmSfaHEgTgUsZzuJ1bAxu_7WlSN1uGlWhLdUC_c9-FaZTV6oXoKaLcjAuLLFG6L8DY5Zv-BpLPfdtseKoq-O636HyBMrhQN7YjgAsXrI2GEgDC91BLJ-lhbCwkLeNP1KKgreKFM8dlO95KHs6YpvlsjQMEdL1zoDC8EIV9t3C91VvLr72GLydlJCHws_Qq0KgXB6zeaBBYSNl5aZoYghAO1XwOgmQ3HT92GPIrBAkIaw5rcl9T0y3y8dQTkWcoM9kAkZDJ9DKzMS-IIvRoyiUKOjJnkgCx-UrQtzWEioz-5n9uHzLEQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.9834715,
                    "lng": 27.9510976
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.98171747010728,
                        "lng": 27.95267107989272
                    },
                    "southwest": {
                        "lat": -32.9844

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJx7LomfiGIZqyy9liPkRezlMvzwEMY3R0fBjeSGoI0vim3SfkFgkZCPVp-8MPez5o362NpEJq4CyF8F1nenBiwFIHCTP2tQHwcXmfeCuHZXxNzatZC2nO5rZ_d4X_5odwgsOi9xKs1hjqiRyEU3yH4JpoDl2J0rPoqE1Q2PcQnIxFsEddy1IY-u95Nc-Dt1giGQf5OiVdNlCpfZIPajVnSDPUyfRLERpeVJznrcb5lymBFc1pINnLrTD71rRENjqL-cIr3g5Ne0AkCpN-t232ksFGnvR-_0oQK6QsbJ7yVhQWLXnTcXLS4LOW5EU5Hbof_dyXJOb1FQ0WKyO3nFzh1huyTOJ_1Z3xfoO1k3bzHUVCwUB48RVdCq7wuzbq_ltBF34503_V3SPAQota7FSUv2TKMUgfc1ZQYpodyxrU3nOZjKQnzvz5xuNQFsweWjQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.0042877,
                    "lng": 54.0709533
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.00598597989272,
                        "lng": 54.07226542989272
                    },
                    "southwest": {
                        "lat": 17.0032863

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -36.8921881,
                    "lng": -60.321769
                },
                "viewport": {
                    "northeast": {
                        "lat": -36.89079852010728,
                        "lng": -60.32046172010728
                    },
                    "southwest": {
                        "lat": -36.89349817989272,
                        "lng": -60.32316137989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Santa Rosa",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 490,
                    "html_attributions": [
                        "<

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.6881437,
                    "lng": -53.4465252
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.68670522010728,
                        "lng": -53.44534982010727
                    },
                    "southwest": {
                        "lat": -33.68940487989273,
                        "lng": -53.44804947989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bertelli Chu\u00ed Hotel",
            "photos": [
                {
                    "height": 2268,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/117258703733313923022\"

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKfmtCPZ7RdDK4Y8saPJFvY4kOl5QGQF9SobUUm3GrDEn-AF9rbUgSvOFOH7DpnHGyzmY9OBG2YfOE31hO4Ll3-FSSOOpIMFssCThIpXwBUr4vFXwsOaQ5nu-fwKrBmA0Yki07wRThdv6CNbLimzxY_XSAR1b3SQOny_1BsKtfFMv-Urw2ZBWvDXAkPJMijuvYB54wio58lWy9zJ_c_s24u6GJ_Z4GaoW8kFb3qoQxCPpjpUaLQuDYngJoVbsZ62lNPAEeU7OUybVnI2ZiTgw3Yyi3D3CDAeFQcFJqBTHKRFX2WW0f3ELxMm3X11h_WSRcga3ziZ_QNMd4hhCf9RubtPg_2IcDWJTgnTQ43WdZio-L2lBIz-cjYZ8UWXJcs9q_q_Y6TfsdyeihyGlz7qH4wN9R3muT7tsNH8xu4s4VPbEDS8dQDDVVzD-ha9s-_og",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.95777949999999,
                    "lng": 141.4657271
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.95655232010728,
                        "lng": 141.4671747298927
                    },
                    "southwest": {
                        "lat": 

In [10]:
hotel_df

,City,Max Temp,Cloudiness,Latitude,Longitude,Humidity,Wind Speed,Country,Date,Hotel Name
38,east london,72.03,0,-33.0153,27.9116,77,3.96,ZA,1612904371,Blue Lagoon Hotel
72,salalah,75.20,0,17.0151,54.0924,60,1.14,OM,1612904380,Star Emirates garden villas
75,olavarria,75.00,0,-36.8927,-60.3225,45,5.99,AR,1612904380,Hotel Santa Rosa
154,chuy,72.46,0,-33.6971,-53.4616,75,9.10,UY,1612904394,Bertelli Chuí Hotel
415,broken hill,71.60,0,-31.9500,141.4333,28,6.91,AU,1612904491,Royal Exchange Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
fig_layout={
    'Width':'900px',
    'Height':'600px',
    'Border':'1px solid black',
    'Padding':'1px',
    'Margin':'0 auto 0 auto'
}
final_fig=gmaps.figure(layout=fig_layout,zoom_level=3,center=(30,40))

markers=gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
final_fig.add_layer(heat_layer)
final_fig.add_layer(markers)

# Display figure
final_fig


Figure(layout=FigureLayout(height='420px'))